# Optimized ZORRO Data Cleaner Experiments

In [1]:
import os
import copy
import pickle
import sympy
import symengine as se
import functools
import itertools
import psutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from error_injection import MissingValueError, SamplingError, Injector
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import mutual_info_score, auc, roc_curve, roc_auc_score, f1_score
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize as scipy_min
from scipy.spatial import ConvexHull
from scipy.optimize import minimize, Bounds, linprog
from sympy import Symbol as sb
from sympy import lambdify
from tqdm.notebook import trange,tqdm
from IPython.display import display,clear_output
from random import choice

import cProfile
import time
import pstats

from dowhy import CausalModel
from dowhy import causal_estimators
import dowhy.datasets

import ginac_module
import yep
import sys
from multiprocessing import Pool, cpu_count
import math
import regex
import string
from functools import partial

np.random.seed(1)

In [2]:
profiler = cProfile.Profile()

In [3]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
def create_symbol(suffix=''):
    global symbol_id
    symbol_id += 1
    name = f'e{symbol_id}_{suffix}' if suffix else f'e{symbol_id}'
    return sympy.Symbol(name=name)

In [5]:
# helper functions from causal inference not appeared in zono_reg library

symbol_id = -1
scaler_symbols = set([sb(f'k{i}') for i in range(100)])
linearization_dict = dict()
reverse_linearization_dict = dict()


def compute_closed_form(X, y):
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))

# def sample_data(imputed_datasets, uncert_inds=[], seed=42):
#     imp_np = np.array(imputed_datasets)
#     if len(uncert_inds) == 0:
#         uncert_inds = list(itertools.product(range(imp_np.shape[1]),range(imp_np.shape[2])))
#     np.random.seed(seed)
#     choices = np.random.choice(np.arange(imp_np.shape[0]), len(uncert_inds), replace=True)
#     sample_result = imputed_datasets[0].copy()
#     for i, ind in enumerate(uncert_inds):
#         sample_result[ind[0]][ind[1]] = imputed_datasets[choices[i]][ind[0]][ind[1]]
#     return sample_result

def sample_data(uncert_inds, uncertain_attr, X_extended_max, X_extended_min, seed=42):
    np.random.seed(seed)
    sample_result = []
    for u in uncert_inds:
        maximum = X_extended_max[u, uncertain_attr]
        minimum = X_extended_min[u, uncertain_attr]
        sample_result.append(np.random.uniform(minimum, maximum))
    return sample_result

# X_extended_max[:, uncertain_attr]


def linearization(expr_ls):
    # processed_expr_ls = [0 for _ in range(len(expr_ls))]
    processed_expr_ls = []
    print(len(expr_ls))
    for expr_id, expr in enumerate(expr_ls):
        print("Linearization", expr_id)
        # Do not support monomial expr currently, e.g., expr = 1.5*e1. 
        # At lease two monomials in expr, e.g., expr = 1.5*e1 + 2.
        if not(expr.free_symbols):
            processed_expr_ls[expr_id] += expr
            continue
        expr = expr.expand()
        processed_expr_ls.append(expr)
        # for arg in expr.args:
        #     print(arg)
        #     if not(arg.free_symbols):
        #         processed_expr_ls[expr_id] += arg
        #         continue
        #     p = arg.as_poly()
        #     monomial_exponents = p.monoms()[0]
            
        #     # only deal with non-linear monomials (order > 2)
        #     if sum(monomial_exponents) <= 1:
        #         processed_expr_ls[expr_id] += arg
        #         continue

        #     monomial = sympy.prod(x**k for x, k in zip(p.gens, monomial_exponents) 
        #                           if not(x in scaler_symbols))
        #     # check global substitution dictionary
        #     if monomial in linearization_dict:
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*linearization_dict[monomial]
        #     else:
        #         print("not in dict")
        #         found = False
        #         subs_monomial = create_symbol()
        #         for symb in monomial.free_symbols:
        #             if symb in reverse_linearization_dict:
        #                 equivalent_monomial = monomial.subs(symb, reverse_linearization_dict[symb])
        #                 if equivalent_monomial in linearization_dict:
        #                     subs_monomial = linearization_dict[equivalent_monomial]
        #                     found = True
        #                     break
        #         linearization_dict[monomial] = subs_monomial
        #         if not(found):
        #             reverse_linearization_dict[subs_monomial] = monomial
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*subs_monomial
                
    return processed_expr_ls


def merge_small_components_pca(expr_ls, budget=10):
    if not(isinstance(expr_ls, sympy.Expr)):
        expr_ls = sympy.Matrix(expr_ls)
    if expr_ls.free_symbols:
        center = expr_ls.subs(dict([(symb, 0) for symb in expr_ls.free_symbols]))
    else:
        return expr_ls
    monomials_dict = get_generators(expr_ls)
    generators = np.array([monomials_dict[m] for m in monomials_dict])
    if len(generators) <= budget:
        return expr_ls
    monomials = [m for m in monomials_dict]
    pca = PCA(n_components=len(generators[0]))
    pca.fit(np.concatenate([generators, -generators]))
    transformed_generators = pca.transform(generators)
    transformed_generator_norms = np.linalg.norm(transformed_generators, axis=1, ord=2)
    # from largest to lowest norm
    sorted_indices = transformed_generator_norms.argsort()[::-1].astype(int)
    sorted_transformed_generators = transformed_generators[sorted_indices]
    sorted_monomials = [monomials[idx] for idx in sorted_indices]
    new_transformed_generators = np.concatenate([sorted_transformed_generators[:budget], 
                                                 np.diag(np.sum(np.abs(sorted_transformed_generators[budget:]), 
                                                                axis=0))])
    new_generators = pca.inverse_transform(new_transformed_generators)
    new_monomials = sorted_monomials[:budget] + [create_symbol() for _ in range(len(generators[0]))]
    
    processed_expr_ls = center
    for monomial_id in range(len(new_monomials)):
        processed_expr_ls += sympy.Matrix(new_generators[monomial_id])*new_monomials[monomial_id]
    
    return processed_expr_ls


def get_vertices(affset):
    l = len(affset)
    distinct_symbols = set()
    for expr in affset:
        if not(isinstance(expr, sympy.Expr)):
            assert isinstance(expr, int) or isinstance(expr, float)
        else:
            if distinct_symbols:
                distinct_symbols = distinct_symbols.union(expr.free_symbols)
            else:
                distinct_symbols = expr.free_symbols
    distinct_symbols = list(distinct_symbols)
    # print(distinct_symbols)
    combs = [list(zip(distinct_symbols,list(l))) for l in list(itertools.product([-1, 1], repeat=len(distinct_symbols)))]
    res = set()
    for assignment in combs:
        res.add(tuple([expr.subs(assignment) for expr in affset]))
    return(res)

In [6]:
def sympy_to_ginac_format(expr):
    """Convert SymPy expression to GiNaC-parseable string format."""
    # Replace SymPy-specific functions with GiNaC equivalents
    s = str(expr)
    s = s.replace('**', '^')  # Exponentiation
    s = s.replace('sin', 'GiNaC::sin')
    s = s.replace('cos', 'GiNaC::cos')
    # Add more replacements as needed
    return s

In [7]:
def sympy_matrix_to_ginac(matrix):
    """Convert SymPy matrix to GiNaC-parseable list of string format."""
    flattened_list = list(matrix.flat())
    flattened_list = [sympy_to_ginac_format(expr) for expr in flattened_list]
    return flattened_list

In [8]:
def ginac_matrix_to_sympy(matrix, rows, cols, symbols_dict):
    POWER_PATTERN = regex.compile(r'\^')
    matrix_list = []
    math_funcs = {
        'sin': sympy.sin, 
        'cos': sympy.cos, 
        'tan': sympy.tan,
        'exp': sympy.exp, 
        'log': sympy.log, 
        'sqrt': sympy.sqrt
    }
    for i in range(rows):
        for j in range(cols):
            idx = i * cols + j
            print(f"converting entry {idx} back to SymPy")
            expr_str = POWER_PATTERN.sub('**', matrix[idx])
            symengine_expr = se.S(expr_str)
            matrix_list.append(symengine_expr)
            # row.append(eval(expr_str, {"__builtins__": {}}, {**symbols_dict, **math_funcs}))

        # matrix_list.append(row)

    se_matrix = se.DenseMatrix(rows, cols, matrix_list)

    print("converting to sympy matrix")
    return sympy.Matrix(rows, cols, [sympy.sympify(se_matrix[i, j]) 
                                for i in range(rows) 
                                for j in range(cols)])

In [9]:
def parse_ginac_tokens(ginac_expr_str, sympy_symbols):
    """
    A string-based approach to convert GiNaC expressions to SymPy
    without any recursion, with support for scientific notation
    
    Parameters:
    -----------
    ginac_expr_str : str
        String representation of a GiNaC expression
    sympy_symbols : dict
        Dictionary mapping symbol names to SymPy symbols
        
    Returns:
    --------
    sympy.Expr
        The equivalent SymPy expression
    """
    # Start timing for performance monitoring
    start_time = time.time()
    
    # Create a case-sensitive symbol map to ensure accurate matching
    symbol_namespace = {}
    for i, (name, symbol) in enumerate(sympy_symbols.items()):
        symbol_namespace[f"sym_{i}"] = symbol
        
    # Add special values to the namespace
    symbol_namespace["nan"] = sympy.nan  # Handle 'nan' as sympy.nan
    
    # Create a reverse mapping for symbol replacement
    symbol_replacements = {}
    for i, name in enumerate(sympy_symbols.keys()):
        # Use word boundaries to ensure we only replace whole symbols
        symbol_replacements[r'\b' + regex.escape(name) + r'\b'] = f"sym_{i}"
    
    # Add special case for 'nan' (must be a whole word)
    # This should NOT be replaced if it's part of a scientific notation
    symbol_replacements[r'\bnan\b'] = "nan"  # Keep 'nan' as is since it's in namespace
    
    # Sort replacements by length (longest first) to avoid substring issues
    sorted_replacements = sorted(
        symbol_replacements.items(), 
        key=lambda x: len(x[0]), 
        reverse=True
    )
    
    print(f"Prepared symbol mappings in {time.time() - start_time:.2f} seconds")
    
    # Split the expression into manageable chunks at + and - operators
    chunks = []
    current_chunk = ""
    i = 0
    
    # Use a faster single-pass approach for the initial split
    while i < len(ginac_expr_str):
        char = ginac_expr_str[i]
        
        # Check if this is scientific notation
        is_sci_notation = False
        if char in ['+', '-'] and i > 0:
            # Look back to see if this is part of scientific notation
            if ginac_expr_str[i-1].lower() == 'e' and i >= 2 and ginac_expr_str[i-2].isdigit():
                is_sci_notation = True
        
        # Handle plus/minus at top level (not in scientific notation)
        if char in ['+', '-'] and not is_sci_notation:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = char if char == '-' else ''
        else:
            current_chunk += char
        
        i += 1
    
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)
    
    print(f"Split into {len(chunks)} chunks in {time.time() - start_time:.2f} seconds")
    
    # Process chunks in parallel
    chunk_size = 1000  # Adjust based on your system
    batches = [chunks[i:i + chunk_size] for i in range(0, len(chunks), chunk_size)]
    
    results = []
    for batch_idx, batch in enumerate(batches):
        batch_start = time.time()
        
        # Process this batch in parallel
        n_processes = min(cpu_count(), len(batch))
        if n_processes > 1:
            with Pool(processes=n_processes) as pool:
                process_func = partial(
                    process_term, 
                    sorted_replacements=sorted_replacements,
                    symbol_namespace=symbol_namespace
                )
                batch_results = pool.map(process_func, batch)
        else:
            # For small batches, avoid Pool overhead
            batch_results = [
                process_term(
                    term, 
                    sorted_replacements=sorted_replacements,
                    symbol_namespace=symbol_namespace
                ) 
                for term in batch
            ]
        
        # Combine results from this batch
        batch_result = sum(batch_results)
        results.append(batch_result)
        
        print(f"Processed batch {batch_idx+1}/{len(batches)} in {time.time() - batch_start:.2f} seconds")
    
    # Combine all batch results
    final_result = sum(results)
    print(f"Total processing time: {time.time() - start_time:.2f} seconds")
    
    return final_result

def process_term(term, sorted_replacements, symbol_namespace):
    """
    Process a single term from the expression.
    Uses pre-computed symbol replacements for consistent handling.
    """
    # Handle scientific notation and exponentiation
    term = term.replace("^", "**")
    
    # Process scientific notation in parentheses first
    processed_term = ""
    i = 0
    
    while i < len(term):
        # Scientific notation in parentheses (e.g., (6.741051087966850411E-11))
        if i < len(term) - 1 and term[i] == '(' and term[i+1].isdigit():
            # Find the end of the parenthesized scientific notation
            end_paren = term.find(')', i)
            if end_paren != -1:
                sci_notation = term[i+1:end_paren]
                # Convert to SymPy Float for precision
                processed_term += f"sympy.Float('{sci_notation}')"
                i = end_paren + 1
                continue
        
        # Regular character
        processed_term += term[i]
        i += 1
    
    # Handle special constants before symbol replacements
    # Replace 'nan' that is not part of a scientific notation with sympy.nan
    processed_term = regex.sub(r'\bnan\b', 'sympy.nan', processed_term)
    
    # Now apply all symbol replacements in order (longest first)
    for pattern, replacement in sorted_replacements:
        processed_term = regex.sub(pattern, replacement, processed_term)
    
    # Create namespace for evaluation
    namespace = {
        "sympy": sympy,
        **symbol_namespace
    }
    
    try:
        # Compile to bytecode first
        compiled_expr = compile(processed_term, '<string>', 'eval')
        return eval(compiled_expr, {"__builtins__": {}}, namespace)
    except Exception as e:
        # More detailed error reporting
        print(f"Error processing term (first 50 chars): '{term[:50]}...': {e}")
        print(f"Processed term was: '{processed_term[:100]}...'")
        # Return zero for this term but continue processing
        return sympy.S.Zero

# Helper function to check for other special mathematical constants
def check_for_special_constants(expr_str):
    """
    Identify special mathematical constants in the expression
    """
    constants = {
        'nan': 'Not a number',
        'inf': 'Infinity',
        'pi': 'Pi constant',
        'e': 'Euler\'s number',
        # Add other constants if needed
    }
    
    found = {}
    for const in constants:
        pattern = r'\b' + const + r'\b'
        if regex.search(pattern, expr_str):
            found[const] = constants[const]
    
    if found:
        print("Found special constants:")
        for const, desc in found.items():
            print(f"  - {const}: {desc}")
    
    return found

# Optional: Add this debugging function to help identify symbol issues
def debug_symbol_matching(expr_str, sympy_symbols):
    """Debug which symbols are found and which are missing"""
    # Find all potential symbol patterns
    symbol_pattern = r'\be[a-zA-Z]?[0-9]+\b'
    found_symbols = set(regex.findall(symbol_pattern, expr_str))
    
    # Check which ones are in the provided dictionary
    missing = [s for s in found_symbols if s not in sympy_symbols]
    present = [s for s in found_symbols if s in sympy_symbols]
    
    print(f"Total potential symbols found: {len(found_symbols)}")
    print(f"Symbols in dictionary: {len(present)}")
    print(f"Symbols not in dictionary: {len(missing)}")
    
    if missing:
        print(f"First 10 missing symbols: {missing[:10]}")
    
    # Check for case-sensitivity issues
    sympy_keys_lower = [k.lower() for k in sympy_symbols.keys()]
    case_issues = [s for s in missing if s.lower() in sympy_keys_lower]
    
    if case_issues:
        print(f"Possible case-sensitivity issues: {case_issues[:10]}")

# Additional utility function for memory tracking (optional)
def get_memory_usage():
    """Return current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

In [10]:
def zorro(ss, X_train, y_train, X_test, y_test, robustness_radius, uncertain_attr, 
                                 uncertain_num, uncertain_radius=None, uncertain_radius_ratio=None, 
                                 lr=0.03, reg=0, seed=42):
    start_time = time.perf_counter()
    X = copy.deepcopy(X_train)
    y = copy.deepcopy(y_train)
    
    symbol_id = -1
    symbolic_data = X.tolist()
    symbols_in_data = set()
    symbol_to_position = dict()
    symbols_dict = dict()
    XS = X.tolist()
    XR = X.tolist()
    for row in range(len(symbolic_data)):
        for col in range(len(symbolic_data[0])):
            xmin = X_extended_min[row][col]
            xmax = X_extended_max[row][col]
            if xmin != xmax:
                xmean = (xmax + xmin) / 2
                xradius = (xmax - xmin) / 2
                new_symbol = create_symbol()
                symbolic_data[row][col] = xmean + xradius*new_symbol
                XS[row][col] = xradius*new_symbol
                XR[row][col] = xmean
                symbols_in_data.add(new_symbol)
                symbol_to_position[new_symbol] = (row, col)
                symbols_dict[str(new_symbol)] = new_symbol
            else:
                XS[row][col] = 0
                
    XS = sympy.Matrix(XS)
    XR = sympy.Matrix(XR)
    n = XS.shape[0]
    n1 = XS.shape[1]
    yR = sympy.Matrix(y_train)
    yS = yR*0.0
    y_test_orig = y_test.copy()
    y_test = sympy.Matrix(y_test.to_numpy().reshape(-1, 1))
    # X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), X_test, axis=1))
    X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), ss.transform(X_test), axis=1))

    # for row in range(n):
    #     for col in range(n1):
    #         expr = copy.deepcopy(XR[row, col])
    #         if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #             XR[row, col] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #             XS[row, col] = expr - XR[row, col]
    #         else:
    #             XR[row, col] = expr
    #             XS[row, col] = 0

    # for row in range(yR.shape[0]):
    #     expr = copy.deepcopy(yR[row])
    #     if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #         yR[row] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #         yS[row] = expr - yR[row]
    #     else:
    #         yR[row] = expr
    #         yS[row] = 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Data loading execution time: {execution_time} seconds")

    start_time = time.perf_counter()
    common_inv = (XR.T*XR + reg*n*np.identity(X.shape[1])).inv()
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Common inverse matrix computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    print("svd base: ", np.array((XR.T*XR).tolist()))
    V, sigma, VT = np.linalg.svd(np.array((XR.T*XR).tolist()).astype(float))
    V = sympy.Matrix(V)
    VT = sympy.Matrix(VT)
    print("VT: ", VT)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"SVD computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wR = common_inv*XR.T*yR
    wS_data = common_inv*((XS.T*XR + XR.T*XS)*wR - XS.T*yR - XR.T*yS)
    wS_non_data = 0.0*VT.row(0).T
    for i in range(X.shape[1]):
        wS_non_data = wS_non_data + sb(f'k{i}')*sb(f'ep{i}')*VT.row(i).T
        symbols_dict[f"ep{i}"] = sb(f'ep{i}')
        symbols_dict[f"k{i}"] = sb(f'k{i}')
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Abstract weights computation time: {execution_time} seconds")
    print(symbols_dict)

    start_time = time.perf_counter()
    eigenvalues = 1 - 2*lr*reg - 2*lr*sigma/n
    for eigenvalue in eigenvalues:
        assert eigenvalue <= 1
        assert eigenvalue >= 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Eigenvalue computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wS = wS_non_data + wS_data
    w = wS + wR
    w_prime = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS + XS.T*XS*wR - XS.T*yS).expand()
    print("w_prime[0]: ", w_prime[0])
    print()
    # w_prime = (XS.T*XR + XR.T*XS + XS.T*XS)*wS + XS.T*XS*wR - XS.T*yS
    # print("Before expansion:", w_prime.rows, w_prime.cols)
    # w_prime = ginac_matrix_to_sympy(ginac_module.expand_matrix(sympy_matrix_to_ginac(w_prime), w_prime.rows, w_prime.cols), w_prime.rows, w_prime.cols, symbols_dict)
    # print("After expansion:", w_prime.rows, w_prime.cols)
    # w_prime = (-lr*2/n)*w_prime
    # print("w_prime final:", w_prime.rows, w_prime.cols, "\n")
    
    coeff_cont = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS_non_data).expand()
    # coeff_cont = (XS.T*XR + XR.T*XS + XS.T*XS)*wS_non_data
    # print("Before expansion:", coeff_cont.rows, coeff_cont.cols)
    # coeff_cont = ginac_matrix_to_sympy(ginac_module.expand_matrix(sympy_matrix_to_ginac(coeff_cont), coeff_cont.rows, coeff_cont.cols), coeff_cont.rows, coeff_cont.cols, symbols_dict)
    # print("After expansion:", coeff_cont.rows, coeff_cont.cols)
    # coeff_cont = (-lr*2/n)*coeff_cont
    # print("coeff_cont final:", coeff_cont.rows, coeff_cont.cols)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"W# (abstract gradient descent?) computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    w_prime_projected = VT*w_prime
    print("w_prime_projected[2]: ", w_prime_projected[2])
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Projection computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    eqs = []
    for d in tqdm(range(X.shape[1]), desc='Equation'):
        eq1 = (1-abs(eigenvalues[d]))*sb(f'k{d}')
        eq2 = 0
        coef_dict = dict()
        coef_dict['const'] = 0
            
        for i in range(X.shape[1]):
            coef_dict[sb(f'k{i}')] = 0
        print(len(w_prime_projected[d].args))
        for arg in w_prime_projected[d].args:
            contain_k = False
            for i in range(X.shape[1]):
                symb_k = sb(f'k{i}')
                if symb_k in arg.free_symbols:
                    # if d == 0:
                        # print(f"{symb_k} in, Terms: {arg.args}")
                    # print("has k:", symb_k, d, arg.args[0], arg.args[1])
                    coef_dict[symb_k] = coef_dict[symb_k] + abs(arg.args[0])
                    contain_k = True
                    break
            if not(contain_k):
                # if d == 0:
                #     print(f"no k, Terms: {arg.args}")
                coef_dict['const'] = coef_dict['const'] + abs(arg.args[0])
        eq2 = coef_dict['const']
        for i in range(X.shape[1]):
            eq2 = eq2 + sb(f'k{i}')*coef_dict[sb(f'k{i}')]

        print(eq1, "=", eq2)
        eqs.append(sympy.Eq(eq1, eq2))

    # print()
    # print(eqs)
    result = sympy.solve(eqs, [sb(f'k{i}') for i in range(X.shape[1])])
    print(result)
    for ki in result:
        assert result[ki] >= 0
    param = wR + wS.subs(result)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Equation solving time: {execution_time} seconds")
    
    # center_preds = X_test*get_expr_center(param)
    # print('one-poss-world:', [((center_preds - y_test).T*(center_preds - y_test)/n)[0]])
    start_time = time.perf_counter()
    test_preds = X_test*param
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Prediction time: {execution_time} seconds")
    # print(test_preds[0])
    # robustness_ls = []
#     se_min_ls = []
#     se_max_ls = []
    # acc_count_low = 0
    # acc_count_high = 0
    # for pred_id in tqdm(range(len(test_preds)), desc='Testing'):
    #     pred = test_preds[pred_id]
    #     label_test = y_test[pred_id]
    #     pred_range_radius = get_expr_range_radius(pred)
    #     if pred_range_radius <= robustness_radius:
    #         robustness_ls.append(1)
    #     else:
    #         robustness_ls.append(0)
    #     pred_center = get_expr_center(pred)
    #     pred_low = pred_center - pred_range_radius
    #     pred_hi = pred_center + pred_range_radius
    #     # get accuracy lower and upper bound
    #     if pred_hi < 0.5 and label_test == 0:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low > 0.5 and label_test == 1:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low < 0.5 and pred_hi > 0.5:
    #         acc_count_high += 1
#         rewrite_pred = pred_center+pred_range_radius*sb('err_pred')
#         se_min, se_max = min_max_sympy_expression((rewrite_pred-label_test)**2)
#         se_min_ls.append(se_min)
#         se_max_ls.append(se_max)
    # preds_diff = (test_preds - y_test).T*(test_preds - y_test)/n
    # start_time = time.perf_counter()
    # ginac_param = sympy_to_ginac_format(param)
    # X_test_list = list(np.array(X_test, dtype=float))
    # X_test_list = [list(l) for l in X_test_list]
    # y_test_list = [float(l[0]) for l in y_test_orig.to_numpy()]
    # ginac_param_list = [sympy_to_ginac_format(s) for s in param]
    # abstract_loss_str = ginac_module.abstract_loss(X_test_list, y_test_list, ginac_param_list)

    # POWER_PATTERN = regex.compile(r'\^')
    # math_funcs = {
    #     'sin': sympy.sin, 
    #     'cos': sympy.cos, 
    #     'tan': sympy.tan,
    #     'exp': sympy.exp, 
    #     'log': sympy.log, 
    #     'sqrt': sympy.sqrt
    # }
    # expr_str = POWER_PATTERN.sub('**', abstract_loss_str[0])
    
    # # preds_diff_full = eval(expr_str, {"__builtins__": {}}, {**symbols_dict, **math_funcs})
    # # compiled_expr = compile(expr_str, '<string>', 'eval')
    # # preds_diff_full = eval(compiled_expr, symbols_dict)
    # preds_diff_full = parse_ginac_tokens(abstract_loss_str[0], symbols_dict)
    # end_time = time.perf_counter()
    # execution_time = end_time - start_time
    # print(f"Loss computation and expansion time: {execution_time} seconds")
    
    
    start_time = time.perf_counter()
    n = X_test.shape[0]
    preds_diff = ((test_preds - y_test).T*(test_preds - y_test)/n)[0]
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Loss computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    preds_diff_full = linearization([preds_diff])[0]
    # preds_diff_center = get_expr_center(preds_diff)
    # preds_diff_radius = get_expr_range_radius(preds_diff)
#     print(preds_diff_center)
#     print(preds_diff_radius)
    # print("Robustness Ratio: " + str(np.mean(robustness_ls)))
    
#     print(param)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Loss linearization time: {execution_time} seconds")
    return result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff, preds_diff_full, X_test

In [11]:
def get_expr_range_radius(expr):
    expr_range_radius = 0
    for arg in expr.args:
        if arg.free_symbols:
            expr_range_radius += abs(arg.args[0])
    return expr_range_radius

def get_expr_center(expr):
    return expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))

In [12]:
def count_matrix_terms(matrix):
    total_terms = 0
    
    # Iterate through each element in the matrix
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            element = matrix[i, j]
            
            # Count terms in this element
            if element.is_Add:
                # If the element is an Add expression (has + or - operations)
                total_terms += len(element.as_ordered_terms())
            elif element != 0:
                # If the element is a single term (not zero)
                total_terms += 1
                
    return total_terms

## Step 1: Data Import and Error Injection

**Generate Sample Dataset**

In [13]:
np.random.seed(0)
data = dowhy.datasets.linear_dataset(beta=0.5,
                                     num_common_causes=5,
                                     num_treatments=1,
                                     num_samples=100,
                                     treatment_is_binary=True,
                                     outcome_is_binary=False,
                                     stddev_treatment_noise=10,
                                    )
df = data["df"]
df['v0'] = df['v0'].astype(float)
df.corr()

,W0,W1,W2,W3,W4,v0,y
W0,1.000000,0.166709,0.067305,0.149616,-0.000326,0.205963,0.432444
W1,0.166709,1.000000,-0.012357,0.018416,0.097455,0.109692,0.398079
W2,0.067305,-0.012357,1.000000,0.048398,0.080499,0.025124,0.614620
W3,0.149616,0.018416,0.048398,1.000000,-0.193899,-0.034596,0.216934
W4,-0.000326,0.097455,0.080499,-0.193899,1.000000,0.081351,0.565829
v0,0.205963,0.109692,0.025124,-0.034596,0.081351,1.000000,0.375317
y,0.432444,0.398079,0.614620,0.216934,0.565829,0.375317,1.000000


In [14]:
df

,W0,W1,W2,W3,W4,v0,y
0,-0.745097,2.400303,1.471645,-0.416110,2.392510,0.0,2.404861
1,1.178439,0.914691,0.784667,-0.091816,1.257514,1.0,2.167766
2,-0.276845,0.705577,-0.755228,0.466693,-0.119251,1.0,0.306247
3,0.778194,-1.133118,-0.361171,-0.152383,1.361701,0.0,0.414408
4,-0.235430,0.477744,1.668267,1.624795,0.413750,1.0,2.039106
...,...,...,...,...,...,...,...
95,1.351012,0.028154,-1.327489,1.636012,-0.028144,1.0,0.656554
96,0.304177,-0.474616,0.860025,0.551097,-1.807282,0.0,-0.451122
97,-1.287803,0.875010,-0.641235,0.983043,0.234071,0.0,-0.057927
98,0.092493,-0.499761,-1.288547,0.558506,0.069644,0.0,-0.597591


In [15]:
X = df.drop(columns=["y"])
y = df[["y"]]
y

,y
0,2.404861
1,2.167766
2,0.306247
3,0.414408
4,2.039106
...,...
95,0.656554
96,-0.451122
97,-0.057927
98,-0.597591


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = copy.deepcopy(X_train).reset_index(drop=True)
X_test = copy.deepcopy(X_test).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_train

,W0,W1,W2,W3,W4,v0
0,-0.276845,0.705577,-0.755228,0.466693,-0.119251,1.0
1,1.141257,-0.330569,0.120095,-0.700197,-0.623530,0.0
2,-1.287803,0.875010,-0.641235,0.983043,0.234071,0.0
3,-0.288402,-0.800214,-1.752745,-0.428196,0.427108,0.0
4,1.037524,1.036768,0.207288,-0.900394,1.719704,0.0
...,...,...,...,...,...,...
75,0.456877,1.312314,0.410540,1.301130,-2.322114,1.0
76,-1.556944,-0.209344,-0.273893,0.401130,-0.928711,0.0
77,0.112264,-1.176088,0.367516,0.624805,0.677898,1.0
78,-0.256001,-0.310369,-0.469495,-0.043018,0.467111,1.0


**Error Injection and Comparison with Original Dataset**

In [17]:
label_range = y_train.max()-y_train.min()
label_range

y    5.571444
dtype: float64

In [18]:
df = X_train.merge(y_train, left_index=True, right_index=True)
df

,W0,W1,W2,W3,W4,v0,y
0,-0.276845,0.705577,-0.755228,0.466693,-0.119251,1.0,0.306247
1,1.141257,-0.330569,0.120095,-0.700197,-0.623530,0.0,-0.209989
2,-1.287803,0.875010,-0.641235,0.983043,0.234071,0.0,-0.057927
3,-0.288402,-0.800214,-1.752745,-0.428196,0.427108,0.0,-1.091531
4,1.037524,1.036768,0.207288,-0.900394,1.719704,0.0,1.416271
...,...,...,...,...,...,...,...
75,0.456877,1.312314,0.410540,1.301130,-2.322114,1.0,0.332114
76,-1.556944,-0.209344,-0.273893,0.401130,-0.928711,0.0,-1.044334
77,0.112264,-1.176088,0.367516,0.624805,0.677898,1.0,0.909324
78,-0.256001,-0.310369,-0.469495,-0.043018,0.467111,1.0,0.344581


In [19]:
# error injection, 10 in this case

all_cols = ['W0', 'W1', 'W2', 'W3', 'W4', 'v0']
all_cols_idx = [df.columns.to_list().index(c) for c in all_cols]

# num of errors injected
mv_num = 10

# which col to inject missing val
treatment_idx = all_cols.index('v0')
uncertain_attr = all_cols.index('W0')
print(treatment_idx, uncertain_attr)

# MNAR
def non_random_pattern(data_X, data_y):
    binary_indicators = []
    for i in range(data_X.shape[0]):
        if (data_X.iloc[i, treatment_idx] == 1) and (data_X.iloc[i, uncertain_attr] > 0):
            binary_indicators.append(1)
        else:
            binary_indicators.append(0)
    return np.array(binary_indicators)

sample_pattern_len = np.sum(non_random_pattern(df.drop('y', axis=1), df.y))
mv_ratio = mv_num/sample_pattern_len
mv_err = MissingValueError(uncertain_attr, pattern=non_random_pattern, ratio=mv_ratio)
dirty_df, dirty_y, _, _ = mv_err.inject(df.drop('y', axis=1).copy(), df.y.copy(), df.drop('y', axis=1), df.y)

X_extended = np.append(np.ones((len(dirty_df), 1)), 
                       dirty_df.to_numpy().astype(float)[:, all_cols_idx], axis=1)
X_extended_clean = np.append(np.ones((len(df), 1)), 
                             df.to_numpy().astype(float)[:, all_cols_idx], axis=1)

ss = StandardScaler()
X_extended[:, 1:] = ss.fit_transform(X_extended[:, 1:])
X_extended_clean[:, 1:] = ss.transform(X_extended_clean[:, 1:])

param_clean = compute_closed_form(X_extended_clean, y_train)

# first column becomes constant 1's
treatment_idx += 1
uncertain_attr += 1
# remain_col_idx = [0] + [i+1 for i in remain_col_idx]

5 0


In [20]:
injected_size = mv_num

In [21]:
dirty_df

,W0,W1,W2,W3,W4,v0
0,-0.276845,0.705577,-0.755228,0.466693,-0.119251,1.0
1,1.141257,-0.330569,0.120095,-0.700197,-0.623530,0.0
2,-1.287803,0.875010,-0.641235,0.983043,0.234071,0.0
3,-0.288402,-0.800214,-1.752745,-0.428196,0.427108,0.0
4,1.037524,1.036768,0.207288,-0.900394,1.719704,0.0
...,...,...,...,...,...,...
75,NaN,1.312314,0.410540,1.301130,-2.322114,1.0
76,-1.556944,-0.209344,-0.273893,0.401130,-0.928711,0.0
77,NaN,-1.176088,0.367516,0.624805,0.677898,1.0
78,-0.256001,-0.310369,-0.469495,-0.043018,0.467111,1.0


In [22]:
new_df = df.drop(columns=['y'])
df_diff = pd.concat([new_df, dirty_df])
df_diff = df_diff.drop_duplicates(keep=False)
df_diff

# first 10: original data point; last 10: data point with error injected

,W0,W1,W2,W3,W4,v0
13,1.351012,0.028154,-1.327489,1.636012,-0.028144,1.0
15,0.201563,-0.399192,0.698286,0.179879,-1.150258,1.0
23,1.115507,0.771966,-1.045359,1.015017,-1.057477,1.0
26,1.523133,0.658961,-0.052137,0.140137,-1.532901,1.0
31,1.039461,0.148244,-0.770128,0.187953,0.752800,1.0
51,1.621854,2.293954,1.496905,1.390463,0.765512,1.0
63,1.035186,0.175771,1.179125,0.297049,0.946952,1.0
74,0.246892,-0.647899,1.600999,1.877250,-0.722208,1.0
75,0.456877,1.312314,0.410540,1.301130,-2.322114,1.0
77,0.112264,-1.176088,0.367516,0.624805,0.677898,1.0


In [23]:
len(X_extended_clean)

In [24]:
imputers = [KNNImputer(n_neighbors=3), KNNImputer(n_neighbors=5), KNNImputer(n_neighbors=10), IterativeImputer(random_state=42)]
num_attrs = X_extended.shape[1]
X_nan = X_extended.copy()
imputed_cols = [X_extended_clean[:, uncertain_attr]]
# imputed_other = [X_extended_clean[:, other_uncertain]]
imputed_datasets = [X_extended_clean]
for imp in imputers:
    imputed_dataset = imp.fit_transform(X_nan)
    imputed_datasets.append(imputed_dataset)
    imputed_cols.append(imputed_dataset[:, uncertain_attr])
    # imputed_other.append(imputed_dataset[:, other_uncertain])

X_extended_max = X_extended.copy()
X_extended_max[:, uncertain_attr] = np.max(imputed_cols, axis=0)
# X_extended_max[:, other_uncertain] = np.max(imputed_other, axis=0)
# X_extended_max = X_extended_max[:, remain_col_idx]

X_extended_min = X_extended.copy()
X_extended_min[:, uncertain_attr] = np.min(imputed_cols, axis=0)
# X_extended_min[:, other_uncertain] = np.min(imputed_other, axis=0)
# X_extended_min = X_extended_min[:, remain_col_idx]

# X_extended = X_extended[:, remain_col_idx]

In [25]:
X_extended_max[0]

array([ 1.        , -0.29047735,  0.28714709, -0.69811843,  0.22871415,
        0.14414354,  1.22474487])

In [26]:
X_test_extended = np.append(np.ones((len(X_test), 1)), 
                       X_test.to_numpy().astype(float)[:, all_cols_idx], axis=1)
X_test_extended[:, 1:] = ss.transform(X_test_extended[:, 1:])
X_test_extended[0]

array([ 1.        ,  0.23548838,  1.12025801,  0.69586645, -2.85149661,
       -0.24617336,  1.22474487])

In [27]:
X_test_list = [list(l) for l in X_test_extended]
y_test_list = [float(l[0]) for l in y_test.to_numpy()]

In [28]:
X_max_list = [list(l) for l in X_extended_max]
X_min_list = [list(l) for l in X_extended_min]

In [29]:
X_train_list = [list(l) for l in X_extended]
y_train_list = [float(l) for l in dirty_y.to_numpy()]

In [30]:
import csv

In [31]:
with open('X_train_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(X_train_list)

with open('X_test_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(X_test_list)

with open('y_train_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in y_train_list:
        writer.writerow([item])

with open('y_test_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in y_test_list:
        writer.writerow([item])
        
with open('X_max_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(X_max_list)

with open('X_min_toy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(X_min_list)

In [32]:
param_list = ginac_module.zorro(X_train_list, y_train_list, X_max_list, X_min_list, 0.01, 0.0)

          80      4.04224  2.64496e-07 -1.95789e-07 -1.77883e-07  3.35276e-08            0
     4.04224      72.6334      12.6437      6.92981      14.6313     -2.39646      11.1216
 2.64496e-07      12.6437           80      2.50168      2.90094      13.8562      15.9783
-1.95789e-07      6.92981      2.50168           80      2.23966      5.59833     -1.13632
-1.77883e-07      14.6313      2.90094      2.23966           80     -17.7242      7.86682
 3.35276e-08     -2.39646      13.8562      5.59833     -17.7242           80      4.08772
           0      11.1216      15.9783     -1.13632      7.86682      4.08772           80
VT  0.0615808    0.489945    0.547114    0.171423    0.361241   0.0980979    0.536026
 -0.0296825   -0.166552    0.321666   0.0986564   -0.590274    0.711147   0.0634273
  -0.347931   -0.179642    0.146387   -0.832263  -0.0340605  -0.0673912    0.356203
  -0.922533  -0.0312771 -0.00385678    0.366523    0.105621 -0.00286225  -0.0493629
  0.0652954   -0.267214  

## Step 2: ZORRO Model Construction

In [33]:
result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff, preds_diff_full, X_test_matrix = zorro(ss, X_extended, dirty_y, X_test, y_test, 0.05, uncertain_attr, 
                             injected_size, uncertain_radius=None, uncertain_radius_ratio=None, lr=0.01, reg=0, seed=42)

Data loading execution time: 0.03591631800372852 seconds
Common inverse matrix computation time: 0.030867079003655817 seconds
svd base:  [[80.0000000000000 4.04224482069178 -2.66453525910038e-15
  -1.49880108324396e-15 -2.91433543964104e-15 -1.77635683940025e-15
  -4.88498130835069e-15]
 [4.04224482069178 72.6333649053345 12.6437063382491 6.92980972470837
  14.6313421820084 -2.39646262644211 11.1215763487135]
 [-2.66453525910038e-15 12.6437063382491 80.0000000000000
  2.50168261128023 2.90094278394494 13.8562007488659 15.9782927335753]
 [-1.49880108324396e-15 6.92980972470837 2.50168261128023
  80.0000000000000 2.23965638228146 5.59832585267236 -1.13632275812331]
 [-2.91433543964104e-15 14.6313421820084 2.90094278394494
  2.23965638228146 79.9999999999999 -17.7241784060693 7.86681545636035]
 [-1.77635683940025e-15 -2.39646262644211 13.8562007488659
  5.59832585267236 -17.7241784060693 80.0000000000000 4.08771737422593]
 [-4.88498130835069e-15 11.1215763487135 15.9782927335753
  -1.1363

Equation:   0%|          | 0/7 [00:00<?, ?it/s]

295
0.0280401627255028*k0 = 0.00209233878940088*k0 + 0.00079391415045285*k1 + 0.000716343311676962*k2 + 0.000730770259274732*k3 + 0.000611770700889564*k4 + 0.00186368828085972*k5 + 0.000687984997958504*k6 + 0.000190895438383842
295
0.0256703838140897*k1 = 0.00079391415045285*k0 + 0.000470063078756683*k1 + 0.000211636894593924*k2 + 0.000244254252295119*k3 + 0.000386745175730215*k4 + 0.0011942559311219*k5 + 0.000207194779335399*k6 + 6.70499303642079e-5
295
0.0205217673672392*k2 = 0.000716343311676962*k0 + 0.000211636894593924*k1 + 0.000418443396291893*k2 + 0.000248304830473801*k3 + 0.000387200682767081*k4 + 0.000955566531381165*k5 + 0.000251566007452053*k6 + 7.39809587222315e-5
295
0.0200342614589682*k3 = 0.000730770259274732*k0 + 0.000244254252295119*k1 + 0.000248304830473801*k2 + 8.47788899850449e-5*k3 + 0.000334070158030831*k4 + 0.00108309366953398*k5 + 8.15203597463681e-5*k6 + 1.65899529222984e-5
295
0.0158643944644725*k4 = 0.000611770700889564*k0 + 0.000386745175730215*k1 + 0.000387

In [33]:
profiler.enable()

In [34]:
result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff_full, X_test_matrix = zorro(ss, X_extended, dirty_y, X_test, y_test, 0.05, uncertain_attr, 
                             injected_size, uncertain_radius=None, uncertain_radius_ratio=None, lr=0.01, reg=0, seed=42)

Data loading execution time: 0.09542579599656165 seconds
Common inverse matrix computation time: 0.08240699203452095 seconds
svd base:  [[80.0000000000000 4.04224482069178 -2.66453525910038e-15
  -1.49880108324396e-15 -2.91433543964104e-15 -1.77635683940025e-15
  -4.88498130835069e-15]
 [4.04224482069178 72.6333649053345 12.6437063382491 6.92980972470837
  14.6313421820084 -2.39646262644211 11.1215763487135]
 [-2.66453525910038e-15 12.6437063382491 80.0000000000000
  2.50168261128023 2.90094278394494 13.8562007488659 15.9782927335753]
 [-1.49880108324396e-15 6.92980972470837 2.50168261128023
  80.0000000000000 2.23965638228146 5.59832585267236 -1.13632275812331]
 [-2.91433543964104e-15 14.6313421820084 2.90094278394494
  2.23965638228146 79.9999999999999 -17.7241784060693 7.86681545636035]
 [-1.77635683940025e-15 -2.39646262644211 13.8562007488659
  5.59832585267236 -17.7241784060693 80.0000000000000 4.08771737422593]
 [-4.88498130835069e-15 11.1215763487135 15.9782927335753
  -1.13632

Equation:   0%|          | 0/7 [00:00<?, ?it/s]

295
0.0280401627255028*k0 = 0.00209233878940088*k0 + 0.00079391415045285*k1 + 0.000716343311676962*k2 + 0.000730770259274732*k3 + 0.000611770700889564*k4 + 0.00186368828085972*k5 + 0.000687984997958504*k6 + 0.000190895438383842
295
0.0256703838140897*k1 = 0.00079391415045285*k0 + 0.000470063078756683*k1 + 0.000211636894593924*k2 + 0.000244254252295119*k3 + 0.000386745175730215*k4 + 0.0011942559311219*k5 + 0.000207194779335399*k6 + 6.70499303642079e-5
295
0.0205217673672392*k2 = 0.000716343311676962*k0 + 0.000211636894593924*k1 + 0.000418443396291893*k2 + 0.000248304830473801*k3 + 0.000387200682767081*k4 + 0.000955566531381165*k5 + 0.000251566007452053*k6 + 7.39809587222315e-5
295
0.0200342614589682*k3 = 0.000730770259274732*k0 + 0.000244254252295119*k1 + 0.000248304830473801*k2 + 8.47788899850449e-5*k3 + 0.000334070158030831*k4 + 0.00108309366953398*k5 + 8.15203597463681e-5*k6 + 1.65899529222984e-5
295
0.0158643944644725*k4 = 0.000611770700889564*k0 + 0.000386745175730215*k1 + 0.000387

In [35]:
profiler.disable()
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

         23024116 function calls (22035731 primitive calls) in 8.146 seconds

   Ordered by: cumulative time
   List reduced from 2286 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    8.143    4.072 /home/hky/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py:522(process_one)
        2    0.000    0.000    8.143    4.072 /home/hky/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py:386(dispatch_shell)
        2    0.000    0.000    8.140    4.070 /home/hky/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:358(execute_request)
        2    0.000    0.000    8.140    4.070 /home/hky/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py:732(execute_request)
        2    0.000    0.000    8.137    4.069 /home/hky/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:364(do_execute)
        2    0.000    0.000    8.136    4.068 /home/hky/.local/lib/python3.10/site-packag

In [36]:
print(param)

Matrix([[0.00410619966955451*e0 + 0.00078253781600925*e1 + 0.00220019428523851*e2 + 0.00291854552005466*e3 + 0.00294658539149318*e4 + 0.00316467282639826*e5 + 0.00243882607902015*e6 + 0.000944024506152327*e7 + 0.000961938985881944*e8 + 0.000860927620727866*e9 - 0.000631726112322138*ep0 + 0.000138707535570607*ep1 - 0.00202125117749352*ep2 + 0.00290340752442568*ep3 - 0.000689637802310882*ep4 + 0.00408570746208479*ep5 + 3.52626504089937e-5*ep6 + 0.279881813334627], [-0.00349813515075792*e0 - 0.000187023309750974*e1 - 0.000391798735358935*e2 - 7.40430661113355e-5*e3 - 0.000957197865246128*e4 - 0.0019596719288642*e5 + 0.00065366419272026*e6 + 0.000120482136384494*e7 - 0.000510050465368527*e8 - 0.000217321856602142*e9 - 0.00502609907744285*ep0 + 0.000778305124485*ep1 - 0.00104360121881913*ep2 + 9.84353815059206e-5*ep3 + 0.00282226340982263*ep4 - 0.0235137435061473*ep5 - 0.000214833572506179*ep6 + 0.28765151422554], [-0.00287642253252809*e0 - 0.000807538787631407*e1 + 0.000634297194329401*e2 

## Step 3: C++ Pipeline Experiments

In [34]:
ginac_param = sympy_to_ginac_format(param)
X_test_list = list(np.array(X_test_matrix, dtype=float))
X_test_list = [list(l) for l in X_test_list]
y_test_list = [float(l[0]) for l in y_test.to_numpy()]
ginac_param_list = [sympy_to_ginac_format(s) for s in param]

In [35]:
def worker_expand(expression_chunk_list):
    local_sum = sympy.S(0)
    processed_count = 0
    try:
        for expr_str in expression_chunk_list:
            # Call the single-item C++ expansion function
            expanded_str = ginac_module.expand_single(expr_str)

            if expanded_str is None: # Handle potential errors if C++ indicates them
                 print(f"Warning: C++ expansion failed for '{expr_str}'")
                 continue # Skip this expression on error

            # Parse the result string back into a GiNaC expression
            expanded_expr = sympy.sympify(expanded_str)
            # Accumulate locally
            local_sum += expanded_expr
            processed_count += 1

        print(f"Worker finished chunk. Processed {processed_count} items.") # Debug print
        # Serialize the final local sum for this chunk and return it
        return local_sum

    except Exception as e:
        # Log error, potentially including the first expression string for context
        print(f"!!! Error in worker processing chunk starting with '{expression_chunk_list[0]}': {e}")
        # Return an indicator of error
        return "__WORKER_ERROR__"

In [36]:
terms = preds_diff.as_ordered_terms()
terms = [sympy_to_ginac_format(t) for t in terms]
print(terms[0])

2.61251588817504e-5*(-0.828963896197551*e0 + 0.125353922182503*e1 + 0.0399482913915641*e2 + 0.663541509388764*e3 + 0.0715555395722535*e4 + 0.0408628368848411*e5 + 0.178858488911322*e6 - 0.125576505668969*e7 + 0.113772532667471*e8 - 0.0671628648688371*e9 - 0.229519021414721*ep0 - 0.397804768324902*ep1 - 0.0649128716253125*ep2 + 0.143204153417001*ep3 - 0.465021682130743*ep4 - ep5 + 0.49250580482789*ep6 + 0.317696066414422)^2


In [37]:
profile_output = 'cpp_profiling_toy.prof'

In [38]:
profiler.clear()
profiler.enable()
start_time = time.perf_counter()
print(f"Starting C++ profiler, output will be in {profile_output}")
yep.start(profile_output)
result_strings = ginac_module.abstract_loss(X_test_list, y_test_list, ginac_param_list)
yep.stop()
print("C++ Profiling completed")
end_time = time.perf_counter()
profiler.disable()
execution_time = end_time - start_time
print(f"GiNaC time: {execution_time} seconds")

Starting C++ profiler, output will be in cpp_profiling_toy.prof
X: 20*7
y: 20*1
param: 7*1
Time for multiplication: 1 ms
Time for difference: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbo

PROFILE: interrupts/evictions/bytes = 0/0/64


In [39]:
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

         240 function calls in 0.366 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.366    0.046 /home/hky/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        8    0.000    0.000    0.366    0.046 {built-in method builtins.exec}
        1    0.346    0.346    0.346    0.346 /home/hky/.local/lib/python3.10/site-packages/yep.py:56(stop)
        1    0.011    0.011    0.011    0.011 {built-in method ginac_module.abstract_loss}
        1    0.000    0.000    0.007    0.007 /tmp/ipykernel_13293/783118916.py:1(<module>)
        1    0.007    0.007    0.007    0.007 /home/hky/.local/lib/python3.10/site-packages/yep.py:32(start)
        2    0.000    0.000    0.001    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.001    0.000 /home/hky/.local/lib/python3.10/site-packages/ipykernel/iostream.py:655(write)
        4    0.000    0.000 

In [40]:
print("\nTo analyze the C++ profiler results, run these commands in a terminal:")
print(f"google-pprof --gif {sys.executable} {profile_output} > prof.gif")


To analyze the C++ profiler results, run these commands in a terminal:
google-pprof --gif /usr/bin/python3 cpp_profiling_toy.prof > prof.gif


In [41]:
start_time = time.perf_counter()
result_sympy = []
    
for expr_str in result_strings:
    # Parse the GiNaC output to SymPy
    # This would need a proper parser for real use
    expr = sympy.sympify(expr_str)
    # expr = sympy.expand(expr)
    # expr = sympy.collect(expr, list(symbols))
        
    result_sympy.append(expr)

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"SymPy conversion time: {execution_time} seconds")

print("Result from GiNaC:")
for expr in result_sympy:
    print(expr)

SymPy conversion time: 0.015373845999420155 seconds
Result from GiNaC:
0.000158158*e0**2 + 0.000469496*e0 + 7.06474e-6*e1**2 + 0.000114805*e1 + 1.42918e-5*e2**2 + 0.000136911*e2 + 7.51222e-5*e3**2 + 0.000211247*e3 + 6.63134e-5*e4**2 + 0.000421418*e4 + 9.57384e-5*e5**2 + 0.000132539*e5 + 7.85561e-5*e6**2 + 0.00043113*e6 + 1.72179e-5*e7**2 + 0.000172243*e7 + 9.12151e-6*e8**2 + 1.37659e-5*e8 + 1.52936e-5*e9**2 + 0.000196536*e9 + 0.000103345*ep0**2 - 0.000195285*ep0 + 4.53946e-5*ep1**2 - 2.36924e-5*ep1 + 3.79665e-5*ep2**2 - 7.0521e-5*ep2 + 1.12124e-5*ep3**2 + 0.000110692*ep3 + 0.000187146*ep4**2 - 0.000628589*ep4 + 0.000906075*ep5**2 - 0.000491123*ep5 + 3.68298e-5*ep6**2 - 0.000107502*ep6 + 0.00110257


In [42]:
print(result_strings[0])

0.000158158*e0^2 + 0.000469496*e0 + 7.06474e-06*e1^2 + 0.000114805*e1 + 1.42918e-05*e2^2 + 0.000136911*e2 + 7.51222e-05*e3^2 + 0.000211247*e3 + 6.63134e-05*e4^2 + 0.000421418*e4 + 9.57384e-05*e5^2 + 0.000132539*e5 + 7.85561e-05*e6^2 + 0.00043113*e6 + 1.72179e-05*e7^2 + 0.000172243*e7 + 9.12151e-06*e8^2 + 1.37659e-05*e8 + 1.52936e-05*e9^2 + 0.000196536*e9 + 0.000103345*ep0^2 - 0.000195285*ep0 + 4.53946e-05*ep1^2 - 2.36924e-05*ep1 + 3.79665e-05*ep2^2 - 7.0521e-05*ep2 + 1.12124e-05*ep3^2 + 0.000110692*ep3 + 0.000187146*ep4^2 - 0.000628589*ep4 + 0.000906075*ep5^2 - 0.000491123*ep5 + 3.68298e-05*ep6^2 - 0.000107502*ep6 + 0.00110257


In [43]:
print(preds_diff_full)

0.000158157506078466*e0**2 - 1.07528477738871e-6*e0*e1 + 3.9477357313312e-5*e0*e2 - 8.19759205475961e-5*e0*e3 + 8.09756030780398e-5*e0*e4 + 7.96326112043433e-5*e0*e5 + 5.66664643532438e-5*e0*e6 + 6.88281712790096e-5*e0*e7 - 6.61930347960308e-6*e0*e8 + 5.47923842359323e-5*e0*e9 - 4.85879077412043e-5*e0*ep0 + 8.06820099163566e-5*e0*ep1 + 1.36511056810209e-5*e0*ep2 - 1.18866451081373e-5*e0*ep3 - 5.22936854078404e-5*e0*ep4 + 0.000278221923610435*e0*ep5 - 0.000124704092564526*e0*ep6 + 0.000469496462765019*e0 + 7.06474124954263e-6*e1**2 + 6.25973895767502e-6*e1*e2 + 3.57648344271567e-5*e1*e3 + 2.57586790599774e-5*e1*e4 - 1.00436634531961e-5*e1*e5 + 3.28234937352913e-5*e1*e6 + 1.03063879307966e-5*e1*e7 + 3.63336623663702e-6*e1*e8 + 1.32607861078678e-5*e1*e9 + 6.12982140528723e-6*e1*ep0 - 6.80697459543938e-6*e1*ep1 - 1.25063613796169e-5*e1*ep2 + 1.04003851581038e-5*e1*ep3 - 6.76857532086822e-5*e1*ep4 - 8.34088668194393e-5*e1*ep5 + 8.85897874934527e-6*e1*ep6 + 0.000114804882215272*e1 + 1.429179

In [43]:
print(preds_diff)

2.61251588817504e-5*(-0.828963896197551*e0 + 0.125353922182503*e1 + 0.0399482913915641*e2 + 0.663541509388764*e3 + 0.0715555395722535*e4 + 0.0408628368848411*e5 + 0.178858488911322*e6 - 0.125576505668969*e7 + 0.113772532667471*e8 - 0.0671628648688371*e9 - 0.229519021414721*ep0 - 0.397804768324902*ep1 - 0.0649128716253125*ep2 + 0.143204153417001*ep3 - 0.465021682130743*ep4 - ep5 + 0.49250580482789*ep6 + 0.317696066414422)**2 + 0.000131826953142964*(-0.576889183629876*e0 + 0.0683289853187644*e1 + 0.0426253531832455*e2 + 0.437404871596156*e3 - 0.0170335764230375*e4 - 0.245735190285987*e5 - 0.0370895515257913*e6 - 0.128669845391081*e7 + 0.0662657126967341*e8 - 0.077953889388183*e9 + 0.0292557843383442*ep0 - 0.17833632052691*ep1 + 0.0641901191028034*ep2 + 0.0937529383170272*ep3 - 0.197902268092922*ep4 - ep5 + 0.344578894362877*ep6 + 0.00897172738295119)**2 + 1.11112230377964e-5*(-0.527015664434232*e0 + 0.029226447904813*e1 - 0.390860411567808*e2 - 0.209430436641613*e3 + 0.107042988832834*e4

## Step 4: SymPy Stats

In [27]:
preds_diff_full

In [37]:
preds_diff

In [26]:
def count_nodes(expr):
    return sum(1 for _ in sympy.preorder_traversal(expr))

In [28]:
print("Node count for preds_diff_full: " + str(count_nodes(preds_diff_full)))

Node count for preds_diff_full: 682


In [31]:
def get_depth(expr):
    if not expr.args:
        return 0
    
    # Find the maximum depth among all children and add 1 for this level
    return 1 + max(get_depth(arg) for arg in expr.args)

In [32]:
print("Tree depth for preds_diff_full: " + str(get_depth(preds_diff_full)))

Tree depth for preds_diff_full: 3


In [34]:
from pympler import asizeof

In [36]:
print("Memory usage for preds_diff_full: " + str(asizeof.asizeof(preds_diff_full)) + " bytes")

Memory usage for preds_diff_full: 257200 bytes


In [38]:
print("Node count for preds_diff: " + str(count_nodes(preds_diff)))

Node count for preds_diff: 1129


In [39]:
print("Tree depth for preds_diff: " + str(get_depth(preds_diff)))

Tree depth for preds_diff: 5


In [40]:
print("Memory usage for preds_diff: " + str(asizeof.asizeof(preds_diff)) + " bytes")

Memory usage for preds_diff: 492408 bytes


## Step 5: GiNaC Module Stats